<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=500px/>
    <font>Python 2023</font><br/>
    <br/>
    <br/>
    <b style="font-size: 2em">Корутины, async/await, asyncio</b><br/>
    <br/>
    <font>Вадим Мазаев</font><br/>
</center>

Напоминание: IO-bound операции — операции, связанные с длительным ожиданием другого устройства, например, сетевой карты или диска

<center>
<img src="https://blog-assets.risingstack.com/2016/Apr/non_async_blocking_operations_example_in_node_hero-1459856858194.png" alt="io-operations" width=800/>
</center>

### DB operation

In [ ]:
from datetime import date
import pymongo

client = MongoClient('localhost', 27017)

posts = client['web_db']['posts']

# blocks until DB answers
results = posts.find({'author': 'Vadim', 'date': date.today})

### Http request

In [ ]:
import requests

# blocks until site returns response
response = requests.get('http://some.site')

## Asynchronous I/O

<center>
<img src="https://camo.githubusercontent.com/a05fd290b0ad342a6721ca3fc66d7ed65c004fa4/68747470733a2f2f63646e2d696d616765732d312e6d656469756d2e636f6d2f6d61782f313630302f312a36306975674742484d46375050536e2d6664517248512e706e67" alt="sync-vs-async" width=700/>
</center>

Asynchronous I/O — неблокирующяя обработка ввода/вывода, которая позволяет процессу продолжить выполнение не дожидаясь окончания передачи данных.

# Корутины (coroutines)

### с самого начала...

In [1]:
def eager_range(up_to: int) -> list[int]:
    sequence = []
    index = 0
    while index < up_to:
        sequence.append(index)
        index += 1
    return sequence

Начиная с Python 2.2 в языке появились генераторы

In [2]:
from collections.abc import Generator

In [3]:
def lazy_range(up_to: int) -> Generator[int, None, None]:
    index = 0
    while index < up_to:
        yield index
        index += 1

В Python 2.5 вводят метод `send()`

In [4]:
def jumping_range(up_to: int) -> Generator[int, int | None, None]:
    index = 0
    while index < up_to:
        jump = yield index
        if jump is None:
            jump = 1
        index += jump

In [5]:
generator = jumping_range(5)

In [6]:
print('next   :', next(generator))
print('send  2:', generator.send(2))
print('next   :', next(generator))
print('send -1:', generator.send(-1))

next   : 0
send  2: 2
next   : 3
send -1: 2


В Python 3.3 добавляется важный синтаксический сахар `yield from`

In [7]:
def bottom() -> Generator[int, None, int]:
    yield 42
    yield 84
    return 0

In [8]:
def top() -> Generator[int, None, None]:
    value = yield from bottom()
    print(value)

In [9]:
list(top())

0


[42, 84]

Наконец, в Python 3.4 вводят фреймворк `asyncio`

In [10]:
import asyncio
import types

In [11]:
@types.coroutine  # asyncio.coroutine
def countdown(n: int) -> Generator[asyncio.Future, None, None]:
    while n > 0:
        print(n)
        yield from asyncio.sleep(1)
        n -= 1

In [12]:
next(countdown(3))

3


<Future pending>

In [13]:
# to start nested event loop inside jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [14]:
loop = asyncio.get_event_loop()
loop.run_until_complete(countdown(3))

3
2
1


Довольно быстро становится понятно, что в языке теперь есть некоторая путаница между генераторами и корутинами

И уже в Python 3.5 вводят `async/await`, заменив generator-based корутины на встроенные в язык

In [15]:
async def compute(a: int, b: int) -> int:
    print('Compute...')
    await asyncio.sleep(1)
    return a + b

In [16]:
compute(3, 5)

<coroutine object compute at 0x7f44cd1a3920>

In [20]:
next(compute(3, 5))

/tmp/ipykernel_186852/1950243043.py:1: RuntimeWarning: coroutine 'compute' was never awaited
  next(compute(3, 5))


TypeError: 'coroutine' object is not an iterator

In [21]:
next(compute(3, 5).__await__())  # awaitable

Compute...


<Future pending>

In [22]:
loop = asyncio.get_event_loop()
loop.run_until_complete(compute(3, 5))

Compute...


8

А затем в Python 3.6 появится возможность реализовывать асинхронные генераторы

In [23]:
async def ticker(upto: int) -> Generator[int, None, None]:
    for i in range(upto):
        await asyncio.sleep(1)
        yield i

In [24]:
next(ticker(2).__anext__().__await__())

<Future pending>

In [25]:
loop = asyncio.get_event_loop()
loop.run_until_complete(ticker(2).__anext__())  # anext(...) from 3.10

0

# Event Loop

In [26]:
async def compute(a: int, b: int) -> int:
    print('Compute...')
    await asyncio.sleep(1)
    return a + b

In [27]:
async def print_sum(a: int, b: int) -> None:
    result = await compute(a, b)
    print(f'{a} + {b} = {result}')

In [28]:
loop = asyncio.get_event_loop()
loop.run_until_complete(print_sum(1, 2))

Compute...
1 + 2 = 3


<center>
<img src="http://ntoll.org/static/images/tulip_coro.png" alt="event-loop" width=1200/>
</center>

In [ ]:
# Simplified code from
# https://github.com/python/cpython/blob/1d1bb95abdcafe92c771fb3dc4722351b032cc24/Lib/asyncio/tasks.py#L630
async def sleep(delay: float) -> None:
    if delay <= 0:
        await __sleep0()
        return result

    loop = asyncio.get_running_loop()
    future = loop.create_future()
    loop.call_later(
        delay,
        future.set_result,
        None,
    )
    
    await future

In [ ]:
@types.coroutine
def __sleep0() -> Generator[None, None, None]:
    yield

# Современный asyncio

## python 3.9+

## Hello world

In [29]:
async def main() -> None:
    print('Hello ...')
    await asyncio.sleep(1)
    print('... World!')

In [30]:
asyncio.run(main())

Hello ...
... World!


## Шедулинг корутин

In [31]:
async def say_after(delay: int, what: str) -> None:
    print(f'wait for {delay}s')
    await asyncio.sleep(delay)
    print(what)

In [32]:
import time

In [33]:
print(f"started at {time.strftime('%X')}")

await say_after(1, 'hello')
await say_after(2, 'world')

print(f"finished at {time.strftime('%X')}")

started at 15:01:26
wait for 1s
hello
wait for 2s
world
finished at 15:01:29


In [34]:
task1 = asyncio.create_task(say_after(2, 'hello'))
task2 = asyncio.create_task(say_after(1, 'world'))

print(f"started at {time.strftime('%X')}")

await task1
await task2

print(f"finished at {time.strftime('%X')}")

started at 15:01:29
wait for 2s
wait for 1s
world
hello
finished at 15:01:31


Задачи (tasks) используются, чтобы запланировать (schedule)

корутины на выполнение "параллельно"

Когда из корутины создают задачу через `asyncio.create_task()`,

она автоматически запускается на следующем такте event loop'а (run soon)

## asyncio.gather

In [35]:
async def factorial(name: str, number: int) -> None:
    result = 1
    for i in range(2, number + 1):
        print(f'Task {name}: Compute factorial({i})...')
        await asyncio.sleep(1)
        result *= i
    print(f'Task {name}: factorial({number}) = {result}')
    return result

In [36]:
await asyncio.gather(
    factorial('A', 2),
    factorial('B', 3),
    factorial('C', 4),
#     return_exceptions=True,
)

Task A: Compute factorial(2)...
Task B: Compute factorial(2)...
Task C: Compute factorial(2)...
Task A: factorial(2) = 2
Task B: Compute factorial(3)...
Task C: Compute factorial(3)...
Task B: factorial(3) = 6
Task C: Compute factorial(4)...
Task C: factorial(4) = 24


[2, 6, 24]

## asyncio.TaskGroup
#### python 3.11+

In [37]:
async with asyncio.TaskGroup() as tg:
    tg.create_task(factorial('A', 2))
    tg.create_task(factorial('B', 3))
    tg.create_task(factorial('C', 4))
    print('tasks pending to start')
print('tasks complete')

tasks pending to start
Task A: Compute factorial(2)...
Task B: Compute factorial(2)...
Task C: Compute factorial(2)...
Task A: factorial(2) = 2
Task B: Compute factorial(3)...
Task C: Compute factorial(3)...
Task B: factorial(3) = 6
Task C: Compute factorial(4)...
Task C: factorial(4) = 24
tasks complete


## Ожидание & таймауты

In [38]:
async def eternity() -> None:
    await asyncio.sleep(3600)  # 1 hour sleep
    print('yay!')

In [44]:
try:
    await asyncio.wait_for(eternity(), timeout=1)  # wait for at most 1 second
except TimeoutError:
    print('timeout!')

timeout!


## Отмена (cancellation) корутин

In [47]:
async def another_eternity() -> None:
    try:
        await asyncio.sleep(3600)
        print(f'{time.strftime("%X")} yay!')
    finally:
        print(f'{time.strftime("%X")} cancelled')
        await asyncio.sleep(5)

In [48]:
print(f'{time.strftime("%X")} started')
try:
    # Send CancelledError to coro after 1 sec and wait for coro completion
    await asyncio.wait_for(another_eternity(), timeout=1)
except TimeoutError:
    print(f'{time.strftime("%X")} timeout!')
print(f'{time.strftime("%X")} finished')

15:59:15 started
15:59:16 cancelled
15:59:21 timeout!
15:59:21 finished


## Защита (shield) от отмены корутин

In [49]:
async def important_task() -> None:
    print(f'{time.strftime("%X")} important task started')
    await asyncio.sleep(5)
    print(f'{time.strftime("%X")} important task finished')

In [50]:
print(f'{time.strftime("%X")} started')
try:
    await asyncio.wait_for(asyncio.shield(important_task()), timeout=1.0)
except asyncio.TimeoutError:
    print(f'{time.strftime("%X")} timeout!')
print(f'{time.strftime("%X")} finished')

16:14:54 started
16:14:54 important task started
16:14:55 timeout!
16:14:56 finished
16:14:59 important task finished


## as_completed

In [51]:
async def factorial(number: int) -> tuple[int, int]:
    result = 1
    for i in range(2, number + 1):
        await asyncio.sleep(1)
        result *= i
    return number, result

In [52]:
for i, future in enumerate(asyncio.as_completed([factorial(4), factorial(3),
                                                 factorial(5), factorial(2)])):
    number, result = await future
    print(f"Factorial({number}) = {result}")

Factorial(2) = 2
Factorial(3) = 6
Factorial(4) = 24
Factorial(5) = 120


## async with

Асинхронный контекстный менеджер - это контекстный менеджер,

который умеет приостанавливать выполнение в методах

входа и выхода: `__aenter__()`, `__aexit__()`

In [ ]:
lock = asyncio.Lock()

async with lock:
    # access shared state

### aiohttp

In [55]:
import aiohttp

In [56]:
async with aiohttp.ClientSession() as session:
    async with session.get('http://ya.ru') as resp:
        text = await resp.text()
        print(text[:70], '...')

<body></body><script nonce='fd1167015079d0121b861dfe9f3c7747'>var it = ...


In [58]:
import requests

In [59]:
with requests.Session() as session:
    with session.get('http://ya.ru') as resp:
        text = resp.text
        print(text[:70], '...')

<body></body><script nonce='172ca618a0fc5e3b31850ee88eb4c372'>var it = ...


## async for

Асинхронный итерируемый (iterable) объект должен возвращать

асинхронный итератор из его iter-метода (`__aiter__()`).

Асинхронный итератор должен возвращать awaitable-объект,

который возвращает следующее значение итератора при вызове `await`,

либо кидать `StopAsyncIteration` из его next-метода (`__anext__()`).

In [64]:
from collections.abc import Iterator

In [65]:
async def ticker(upto: int) -> Iterator[int]:
    for i in range(upto):
        await asyncio.sleep(1)
        yield i

In [73]:
async for i in ticker(3):
    print(i, end=' ')

0 1 2 

In [69]:
async_iter = aiter(ticker(2))  # 3.10+
async_iter

<async_generator object ticker at 0x7fdac90c4820>

In [70]:
await anext(async_iter)  # 3.10+

0

In [71]:
await anext(async_iter)

1

In [72]:
await anext(async_iter)

StopAsyncIteration: 

### Motor

In [75]:
from motor.motor_asyncio import AsyncIOMotorClient

In [76]:
client = AsyncIOMotorClient('localhost', 27017)

In [77]:
await client.test_database.test_collection.insert_one({'key': 'value'})

In [78]:
async for document in client.test_database.test_collection.find({'key': 'value'}):
    print(document)

{'_id': ObjectId('63860ad828a2dab29b375700'), 'key': 'value'}


## Запуск синхронного кода

In [79]:
def blocking_io() -> None:
    print(f"{time.strftime('%X')} start blocking IO")
    time.sleep(5)
    print(f"{time.strftime('%X')} finished blocking IO")

In [80]:
non_blocking_io = asyncio.to_thread(blocking_io)

In [81]:
print(f"{time.strftime('%X')} start gather")
await asyncio.gather(non_blocking_io, asyncio.sleep(5))
print(f"{time.strftime('%X')} finished gather")

17:37:00 start gather
17:37:00 start blocking IO
17:37:05 finished blocking IO
17:37:05 finished gather


## Debugging asyncio

`$ PYTHONASYNCIODEBUG=1 python asyncio_program_to_debug.py`

https://docs.python.org/3/library/asyncio-dev.html

# Спасибо за внимание!